# PCA Demo

 Vinayak Sharma
  
 This notebook aims to train an autoencoder on univariate timeseries data from EKS Performance metrics for MANY NODES. The training data is 'non-anomalous'. 

 The model and implementation techniques can be found in the following github: https://github.com/emerelte/kad
 
 The research paper that this work is based on can be found here: https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9925210

------

In [3]:
##global variable
timesteps = 12
time_steps = 12
batch_size = 6
n_samples = batch_size*100
features = ['node_cpu_utilization','node_memory_utilization','node_network_total_bytes']

## Imports

In [4]:
# !pip install -r /root/eks-ml-pipeline/modeling/kad/kad/requirements.txt
# # # !pip install keras
# # # !pip install tensorflow

In [5]:
# pip install keras

In [6]:
# pip install keras

In [7]:
# import matplotlib as mpl
# mpl.rcParams.keys()

In [8]:
import importlib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import json
import sys
import statsmodels.api as sm
import os
# import keras
# from keras import layers
# import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from pca_ad_dish_5g import pca_ad_dish_5g

from sklearn.model_selection import train_test_split

import _pickle as cPickle


In [9]:
# import logging
# import warnings
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from tensorflow import keras
# import tensorflow as tf
# from tensorflow.keras import layers
# from matplotlib import pyplot as plt

In [10]:
pd.set_option('display.max_rows', None)

First lets read in our training sample set 

In [11]:
training_df_full = pd.read_parquet('/root/healthy_clusters_node_month.parquet')

In [12]:
training_df_full.head()

,Timestamp,InstanceId,ClusterName,node_cpu_utilization,node_cpu_limit,node_cpu_request,node_cpu_usage_total,node_memory_utilization,node_memory_request,node_memory_limit,node_network_rx_bytes,node_network_rx_dropped,node_network_rx_errors,node_network_rx_packets,node_network_total_bytes,node_network_tx_bytes,node_network_tx_dropped,node_network_tx_errors,node_network_tx_packets
0,1654235914008,i-01470d8d8e7b4fd2f,nk-ndc-eks-cluster-dev-usw2-az1-dev-ncm01,2.905276,4000.0,1266.0,116.211031,9.394841,998244352,16476487680,26133.384568,0.0,0.0,87.334264,56498.252063,30364.867495,0.0,0.0,86.300537
1,1654235917104,i-0b36e8825c482f762,nk-ndc-eks-cluster-dev-usw2-az1-dev-ncm01,1.836838,4000.0,1066.0,73.473529,7.296163,851443712,16476487680,5812.096097,0.0,0.0,22.781011,13940.610609,8128.514512,0.0,0.0,22.814785
2,1654232669947,i-01470d8d8e7b4fd2f,nk-ndc-eks-cluster-dev-usw2-az1-dev-ncm01,2.950336,4000.0,1266.0,118.013421,9.399117,998244352,16476487680,24538.169434,0.0,0.0,86.939803,54629.097665,30090.928231,0.0,0.0,86.904324
3,1654232681444,i-0b36e8825c482f762,nk-ndc-eks-cluster-dev-usw2-az1-dev-ncm01,1.858162,4000.0,1066.0,74.326468,7.295069,851443712,16476487680,5451.981961,0.0,0.0,21.206624,13392.905270,7940.923309,0.0,0.0,21.632175
4,1654235970093,i-093473861e74eaf2d,mt-ndc-eks-cluster-dev-mt-use1,0.942944,16000.0,1266.0,150.871041,4.956222,998244352,65902239744,42229.503040,0.0,0.0,251.934982,90291.404482,48061.901442,0.0,0.0,251.558398


In [13]:
columns_to_keep = ['Timestamp','InstanceId','node_cpu_utilization','node_memory_utilization','node_network_total_bytes']

In [14]:
training_df = training_df_full.drop(training_df_full.columns.difference(columns_to_keep),1, inplace=False)
training_df['Timestamp'] = pd.to_datetime(training_df['Timestamp'], unit='ms')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [15]:
len(training_df)

2636965

In [16]:
training_df = training_df.dropna()

In [17]:
test_df = training_df.copy()
training_df = training_df[training_df.InstanceId != 'i-0b36e8825c482f762']

-----

# TRAIN DATASET: Build out Dataset for training on many nodes

now we are starting to build out a training dataset for two nodes. This will prove our understanding of how to model can generalize across many EC2 instances. 

In [18]:
## for normalization
# scaler = StandardScaler()

scaler = StandardScaler()

In [19]:

instance_dfs =[]
for instance in training_df['InstanceId'].unique():
    instance_dfs.append(training_df[training_df.InstanceId == instance].sort_values(by='Timestamp')\
                        .reset_index(drop=True))

import random 

x_train = np.zeros((n_samples,time_steps,len(features)))
for b in range(n_samples):
    
    ##pick random df, and normalize
    df = random.choice(instance_dfs)
    df = df.drop(columns = ['InstanceId'])
    df = df.set_index('Timestamp')
    df = df.sort_index()
    df[features] = scaler.fit_transform(df[features])
    
    
    
    sample = np.zeros((n_samples,len(features)))
    ##make sure length of df is atleast 40
    first_time = random.choice(range(len(df)-time_steps))
    df.head()
    sample = df[features].iloc[first_time:first_time+time_steps]
    x_train[b] = sample

x_train.shape

(600, 12, 3)

our xtrain shape has the following properties: (sample size, time steps, numFeatures)

-------

# TEST DATASET: Build out Dataset for testing on one node

In [20]:
test_df = test_df[test_df.InstanceId == 'i-0b36e8825c482f762']

In [21]:
test_df.head()

,Timestamp,InstanceId,node_cpu_utilization,node_memory_utilization,node_network_total_bytes
1,2022-06-03 05:58:37.104,i-0b36e8825c482f762,1.836838,7.296163,13940.610609
3,2022-06-03 05:04:41.444,i-0b36e8825c482f762,1.858162,7.295069,13392.905270
9,2022-06-03 05:59:35.385,i-0b36e8825c482f762,1.824157,7.296387,14073.509120
11,2022-06-03 05:05:34.448,i-0b36e8825c482f762,1.821361,7.297928,14987.560707
17,2022-06-03 06:00:39.866,i-0b36e8825c482f762,1.811754,7.299718,13253.130119


In [22]:
## drop the instanceId
test_df = test_df.drop("InstanceId",1, inplace=False)


##set timestamp as the index
test_df = test_df.set_index('Timestamp')

##normalize test_df 
test_df[features] = scaler.fit_transform(test_df[features])


##ensure the data is sorted!!
test_df = test_df.sort_index()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


----

# Model Time

----

### First we train

In [23]:
pca_model = pca_ad_dish_5g(num_of_features=3, number_of_temporal_slices=3, timesteps_per_slice=4, n_modes_to_delete=1)

In [24]:
residuals,ed_errors,encode_decode_maps = pca_model.train(x_train)

In [27]:
print(ed_errors.shape)

(1800, 3)


In [24]:
pca_model.save_vs()

### Then we test

In [25]:
pca_model = pca_ad_dish_5g(num_of_features=3, number_of_temporal_slices=3, timesteps_per_slice=4, n_modes_to_delete=1)

In [26]:
pca_model.load_in_vs('vs.npy')

In [27]:
y_hat_test = []
res_test = []

for i in range(0,len(test_df),timesteps):
    if(i + timesteps < len(test_df)):
        sample_topredict_on = test_df.iloc[i:i+timesteps]
        x_test = np.array(sample_topredict_on.to_numpy())
        x_test = x_test.reshape(1,-1,3)
        y_hat,residuals = pca_model.test(x_test)
        y_hat_test.append(y_hat)
        res_test.append(residuals)


### Visualize Testing

In [29]:
y_hat_test[0].shape

(1, 12, 3)